In [1]:
from JTBrix import  run_session, port

In [2]:
from JTBrix.stimuli import register_video

In [3]:
from JTBrix.utils import find_free_port
from JTBrix.experiment import sequence


In [4]:


free_port = find_free_port()
Q1 = ("Which fruit do you prefer?", "Apple", "Banana", "red", "goldenrod", "static/p.jpeg")
Q2 = ("Which car do you prefer?", "Tesla", "Ford", "blue", "gray", "static/p.jpeg")
answers, times = sequence(free_port, Q1, Q2)
print("Answers:", answers)
print("Times:", times)  


 * Serving Flask app 'JTBrix.experiment.run_session'
 * Debug mode: off


 * Running on http://127.0.0.1:55785
Press CTRL+C to quit
127.0.0.1 - - [02/May/2025 22:14:49] "GET /video HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2025 22:14:49] "GET /static/FB.mp4 HTTP/1.1" 404 -
127.0.0.1 - - [02/May/2025 22:14:49] "GET /favicon.ico HTTP/1.1" 204 -


KeyboardInterrupt: 